In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [22]:
#二维卷积==MLP？
#stage1
def image2embed_naive(image,patch_size,weight):
    #image_size:batch_size*channels*h*w
    #F.unfold()拿出卷积的区域
    patch = F.unfold(image,kernel_size=patch_size,stride=patch_size).transpose(-1,-2) #将图片分块
    print(patch.shape)
    patch_embedding=patch@weight
    return patch_embedding
def image2embed_conv(image,kernel,stride):
    conv_output=F.conv2d(image,kernel,stride=stride) #batch_size*output_channels*output_h*output_w
    batch_size=conv_output.shape[0]
    out_channels=conv_output.shape[1]
    out_h,out_w=conv_output.shape[2],conv_output.shape[3]
    #将oh*ow拉直
    patch_embedding = conv_output.reshape((batch_size,out_channels,out_h*out_w)).transpose(-1,-2)
    return patch_embedding

In [33]:
#test code for image2embedding
batch_size,in_channels,image_h,image_w=1,3,8,8
patch_size=4
model_dim=8
max_num_token=16
num_classes=10
label=torch.randint(10,(batch_size,))
patch_depth=patch_size*patch_size*in_channels
image=torch.randn(batch_size,in_channels,image_h,image_w)
weight=torch.randn(patch_depth,model_dim)
print(weight.shape)
print(image2embed_naive(image,patch_size,weight).shape)
patch_embedding=image2embed_naive(image,patch_size,weight)
#kernel的形状因该是out_channels*in_channels*h*w
kernel=weight.transpose(0,1).reshape((-1,in_channels,patch_size,patch_size))
print(image2embed_conv(image,kernel,patch_size).shape)

torch.Size([48, 8])
torch.Size([1, 4, 48])
torch.Size([1, 4, 8])
torch.Size([1, 4, 48])
torch.Size([1, 4, 8])


In [28]:
#stage2
cls_token_embedding=torch.randn(batch_size,1,model_dim,requires_grad=True)
token_embedding=torch.cat([cls_token_embedding,patch_embedding],dim=1)
position_embedding_table=torch.randn(max_num_token,model_dim,requires_grad=True)
seq_len=token_embedding.shape[1]
position_embedding=torch.tile(position_embedding_table[:seq_len],[token_embedding.shape[0],1,1])
token_embedding+=position_embedding

In [30]:
#用torch.nn.TransformerEncoder(encoder_layer,num_layers,norm=None)
encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim, nhead=8)
transformer_encoder=nn.TransformerEncoder(encoder_layer,num_layers=6)
encoder_output=transformer_encoder(token_embedding)
encoder_output.shape

torch.Size([1, 5, 8])

In [35]:
#
cls_token_output=encoder_output[:,0,:]
cls_head=nn.Linear(model_dim,num_classes)
logits=cls_head(cls_token_output)
loss_fn=nn.CrossEntropyLoss()
loss=loss_fn(logits,label)
print(label)
print(loss)

tensor([4])
tensor(3.2685, grad_fn=<NllLossBackward0>)
